In [1]:
import sys
import time
import os
import subprocess
import math
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, Column 
from scipy.stats import linregress
from scipy import interpolate
from scipy import polyval, polyfit
from scipy import odr
import pylab as py
from matplotlib import gridspec
import sklearn.datasets as ds
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import corner
import emcee
import scipy.optimize as op
from scipy.linalg import cholesky, inv,det
from scipy.optimize import minimize
import random
from astropy.table import Table, Column
from linear_mcmc import *

from redTools import *
from Kcorrect import *

In [2]:
band2 = 'w1'
inFile = 'ESN_HI_catal_all.csv'
scaler, pca = transform(inFile, band1 = 'r', band2 = 'w1')

u = scaler.mean_
s = scaler.scale_
v = scaler.var_
## z = (x-u)/s
##u: mean  s:scale  var=s**2

A = pca.explained_variance_ratio_              # The importance of different PCAs components

pca_inv_data = pca.inverse_transform(np.eye(3)) # coefficients to make PCs from features
p0 = pca_inv_data[0,0]
p1 = pca_inv_data[0,1]
p2 = pca_inv_data[0,2]

print u
print s
print p0, p1, p2

/home/ehsan/PanStarrs/Jan/HI/augment/reddening/redTools.py:55: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  table   = np.genfromtxt(inFile , delimiter=',', filling_values=-1, names=True, dtype=None)


[ 2.45058466  2.13581453 22.7712426 ]
[0.20055701 1.31433117 1.38154385]
0.5269886912388262 0.5897016744047068 -0.6119925281493858


In [3]:
band2 = 'w2'
inFile = 'ESN_HI_catal_all.csv'
scaler, pca = transform(inFile, band1 = 'r', band2 = 'w2')

u = scaler.mean_
s = scaler.scale_
v = scaler.var_
## z = (x-u)/s
##u: mean  s:scale  var=s**2

A = pca.explained_variance_ratio_              # The importance of different PCAs components

pca_inv_data = pca.inverse_transform(np.eye(3)) # coefficients to make PCs from features
p0 = pca_inv_data[0,0]
p1 = pca_inv_data[0,1]
p2 = pca_inv_data[0,2]

print u
print s
print p0, p1, p2

[ 2.46222434  1.65517889 23.27221277]
[0.19210835 1.29385952 1.39016989]
0.518405536719746 0.5964988325689783 -0.6127355402146653


/home/ehsan/PanStarrs/Jan/HI/augment/reddening/redTools.py:85: RuntimeWarning: divide by zero encountered in log10
  table['mu50'] = table[band2]+2.5*np.log10(2.*np.pi*(table['R50_'+band2]*60)**2)-2.5*np.log10(table['Wba'])
/home/ehsan/PanStarrs/Jan/HI/augment/reddening/redTools.py:87: RuntimeWarning: divide by zero encountered in divide
  dWba2 = ((0.1/6./table['R50_'+band2])**2)*(1+table['Wba']**2)
/home/ehsan/PanStarrs/Jan/HI/augment/reddening/redTools.py:89: RuntimeWarning: divide by zero encountered in divide
  table['Emu50']=np.sqrt(c2*(0.1/6./table['R50_'+band2])**2+c2*dWba2/table['Wba']**2+0.05**2)
/home/ehsan/PanStarrs/Jan/HI/augment/reddening/redTools.py:91: RuntimeWarning: divide by zero encountered in divide
  table['EC82'] = (5*np.sqrt(2.)/np.log(10))*(0.1/6./table['R50_'+band2])


In [4]:
band2 = 'w2'
inFile = '../reddening/ESN_HI_catal.csv'
scaler, pca = transform(inFile, band1 = 'r', band2 = 'w2')

u = scaler.mean_
s = scaler.scale_
v = scaler.var_
## z = (x-u)/s
##u: mean  s:scale  var=s**2

A = pca.explained_variance_ratio_              # The importance of different PCAs components

pca_inv_data = pca.inverse_transform(np.eye(3)) # coefficients to make PCs from features
p0 = pca_inv_data[0,0]
p1 = pca_inv_data[0,1]
p2 = pca_inv_data[0,2]

print u
print s
print p0, p1, p2

[ 2.46901576  1.62656097 23.34766946]
[0.17995978 1.14617956 1.38362488]
0.5246405135447814 0.6006789987609001 -0.6032719718295168


In [5]:
inFile  = 'ESN_HI_catal_calib.csv'
tt   = np.genfromtxt(inFile , delimiter=',', filling_values=-1, names=True, dtype=None)
print len(tt['pgc'])

882


/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  


In [6]:
def RFA_predict(features=['g_r', 'r_i', 'i_z', 'pc0'], out='i', max_depth=2000, n_estimators=2000, \
            min_samples_leaf=15, bootstrap=True):
    
    ## loading data for training the Random Forest algorithm
    inFile = '../reddening/ESN_INC_P0_MAG.csv'
    table_regr = np.genfromtxt(inFile , delimiter=',', filling_values=-1, names=True, dtype=None)


    ################################ Feature selection, regression parameters
#     features = ['g_r', 'r_i', 'i_z', 'pc0']
    output   = out + '_w1'
#     max_depth=2000
#     n_estimators=2000
    max_features=len(features)
#     min_samples_leaf=15
#     bootstrap=True
    ################################ Regression (Random Forest)

    regr_w1 = RandomForestRegressor(max_depth=max_depth, n_estimators=n_estimators, \
            max_features=max_features, min_samples_leaf=min_samples_leaf, \
                bootstrap=bootstrap)

    table_all = prepareSamples(table_regr, noTest=True)
    x_all, y_all = ML_data(table_all, features, output)
    regr_w1.fit(x_all, y_all)
    p_y_all_w1  = regr_w1.predict(x_all)
    d = np.abs(p_y_all_w1-y_all)
    tt = 0.6
    indx, = np.where(d<tt)
    X = y_all[indx]
    Xp = p_y_all_w1[indx]
    Y = X-Xp
    m1, b1 = np.polyfit(X,Y, 1)
    
    importances = regr_w1.feature_importances_
    print output, 'Feature Importances: ', importances
    ################################ Feature selection, regression parameters
    output   = out + '_w2'
    regr = RandomForestRegressor(max_depth=max_depth, n_estimators=n_estimators, \
            max_features=max_features, min_samples_leaf=min_samples_leaf, \
                bootstrap=bootstrap)
    x_all, y_all = ML_data(table_all, features, output)
    regr.fit(x_all, y_all)
    p_y_all_w2  = regr.predict(x_all)
    d = np.abs(p_y_all_w2-y_all)
    tt = 0.6
    indx, = np.where(d<tt)
    X = y_all[indx]
    Xp = p_y_all_w2[indx]
    Y = X-Xp
    m2, b2 = np.polyfit(X,Y, 1)
    
    importances = regr.feature_importances_
    print output, 'Feature Importances: ', importances
    ################################ Regression (Random Forest)

#     print m1, b1
#     print m2, b2
    return regr_w1, regr, [m1,m2],[b1,b2]

In [7]:
def predict_w1_w2(table, regr_w1, regr, m, b, features, output):
    a = time.time()

    W1_lst = []
    w1p_lst = []
    W2_lst = []
    w2p_lst = []

    for j in range(len(table['pgc'])):
        
        PGC = table['pgc'][j]
        W1  = table['w1'][j]
        W2  = table['w2'][j]
        INC = table["inc"][j]
        M21 = table["m21"][j]
        M0  = table["logWimx"][j]
        WBA = table["Wbap"][j]
        R50 = table["R50_w2p"][j]     
        
        if table['Sqlt'][j]>0:

            def f(w2):
                return predictor(w2, table, regr, features, output, index=j, m=m[1], b=b[1], useFullPredictions=True)
            ###############################################
            out = solver(f, 7, 21, threshold=0.001) 
            if out==None:
                print 'Not solvable ... !!!', PGC
                w2p = 0
                w1p = 0
            else:
                w2p, N = out
                w2p=w2p[0]       
                g_ = table['g0'][j]
                r_ = table['r0'][j]
                i_ = table['i0'][j]
                z_ = table['z0'][j]
                logWimx, c21w, mu50, P0 = get_PC(w2p, M21, M0, WBA, R50)
                g_-=redCorrect(INC, P0, band1='g', band2='w2')
                r_-=redCorrect(INC, P0, band1='r', band2='w2')
                i_-=redCorrect(INC, P0, band1='i', band2='w2')
                z_-=redCorrect(INC, P0, band1='z', band2='w2')
                outDict = {}
                outDict["g_r"] = g_-r_
                outDict["g_i"] = g_-i_
                outDict["g_z"] = g_-z_
                outDict["r_i"] = r_-i_
                outDict["r_z"] = r_-z_
                outDict["i_z"] = i_-z_ 
                
                outDict["pc0"] = P0
                inList=[]
                for f in features: inList.append(outDict[f])
                x_t = np.asarray([inList])
                i_w1p = (regr_w1.predict(x_t)+b[0])/(1.-m[0]) ### regr_w1.predict(x_t)  ### 
                w1p = i_-i_w1p[0]
        else:
            w2p = 0
            w1p = 0
        
        if table['Wqlt'][j]==0:
            W1=0
            W2=0

        W2_lst.append(W2)
        w2p_lst.append(w2p)
        W1_lst.append(W1)
        w1p_lst.append(w1p)    


        ###############################################

    W2_lst = np.asarray(W2_lst)
    w2p_lst = np.asarray(w2p_lst)
    W1_lst = np.asarray(W1_lst)
    w1p_lst = np.asarray(w1p_lst)


    print output, "Time: ", time.time()-a
    return W2_lst, w2p_lst, W1_lst, w1p_lst

# Processing the Entire Catalog

In [8]:
inFile = 'ESN_HI_catal_all.csv'

table   = np.genfromtxt(inFile , delimiter=',', filling_values=-1, names=True, dtype=None, encoding=None)

mag_g  = table['g']
mag_r  = table['r']
mag_i  = table['i']
mag_z  = table['z']

mag_g_  = table['g_']
mag_r_  = table['r_']
mag_i_  = table['i_']
mag_z_  = table['z_']

table   = np.genfromtxt(inFile , delimiter=',', filling_values=-1, names=True, dtype=None, encoding=None)
table = extinctionCorrect(table)
table = Kcorrection(table)


table['mu50'] = table[band2]+2.5*np.log10(2.*np.pi*(table['R50_'+band2]*60)**2)-2.5*np.log10(table['Wba'])
dWba2 = ((0.1/6./table['R50_'+band2])**2)*(1+table['Wba']**2)
c2 = (2.5/np.log(10))**2
table['Emu50']=np.sqrt(c2*(0.1/6./table['R50_'+band2])**2+c2*dWba2/table['Wba']**2+0.05**2)
table['EC82'] = (5*np.sqrt(2.)/np.log(10))*(0.1/6./table['R50_'+band2])

table['R50_w2p'] = halflight(table["R50_g"], table["R50_r"], table["R50_i"], table["R50_z"])
table['Wbap'] = ba(table["Sba"])

## u0, g0, r0, i0, z0 are already K-/extinction- corrected
table['u0'] = table['u']
table['g0'] = table['g']
table['r0'] = table['r']
table['i0'] = table['i']
table['z0'] = table['z']
table['w10']= table['w1']
table['w20']= table['w2']
table["scaler"] = scaler
table["Cpca"]   = pca

/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log10
/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in subtract
/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in divide
/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in divide
/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in sqrt
/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in divide


In [9]:
print len(table['pgc'])

13535


In [10]:
### file2 has priority
def get_predictions(pgc_ids, file1, file2):
    N = len(pgc_ids)
    # prefix = './OLD/mbEQzero/'
    # prefix = './OLD/mbNOTzero/'
    prefix = './'

    inFile = prefix + file1
    table_   = np.genfromtxt(inFile , delimiter='|', filling_values=-1, names=True, dtype=None, encoding=None)
    pgc_x0=table_['pgc']
    W1m_x0=table_['W1m_']
    W1p_x0=table_['W1p_']
    W2m_x0=table_['W2m_']
    W2p_x0=table_['W2p_']
    
    inFile = prefix + file2
    table_   = np.genfromtxt(inFile , delimiter='|', filling_values=-1, names=True, dtype=None, encoding=None)
    pgc_x_=table_['pgc']
    W1m_x_=table_['W1m_']
    W1p_x_=table_['W1p_']
    W2m_x_=table_['W2m_']
    W2p_x_=table_['W2p_']

    W1m_x=np.zeros(N)
    W1p_x=np.zeros(N)
    W2m_x=np.zeros(N)
    W2p_x=np.zeros(N)
    for i,idd in enumerate(pgc_ids):
        if idd in pgc_x_:
            indx, = np.where(pgc_x_==idd)
            W1m_x[i]=W1m_x_[indx][0]
            W1p_x[i]=W1p_x_[indx][0]    
            W2m_x[i]=W2m_x_[indx][0]
            W2p_x[i]=W2p_x_[indx][0]
        elif idd in pgc_x0:
            indx, = np.where(pgc_x0==idd)
            W1m_x[i]=W1m_x0[indx][0]
            W1p_x[i]=W1p_x0[indx][0]
            W2m_x[i]=W2m_x0[indx][0]
            W2p_x[i]=W2p_x0[indx][0]        

    return W1m_x, W1p_x, W2m_x, W2p_x         

In [11]:
pgc = table['pgc']

file1 = 'corrected_mags_linewidth_all_g.csv'
file2 = 'corrected_mags_linewidth_all_g.add.csv'
W1m_g, W1p_g, W2m_g, W2p_g = get_predictions(pgc, file1, file2)

file1 = 'corrected_mags_linewidth_all_r.csv'
file2 = 'corrected_mags_linewidth_all_r.add.csv'
W1m_r, W1p_r, W2m_r, W2p_r = get_predictions(pgc, file1, file2)

file1 = 'corrected_mags_linewidth_all_i.csv'
file2 = 'corrected_mags_linewidth_all_i.add.csv'
W1m_i, W1p_i, W2m_i, W2p_i = get_predictions(pgc, file1, file2)

file1 = 'corrected_mags_linewidth_all_z.csv'
file2 = 'corrected_mags_linewidth_all_z.add.csv'
W1m_z, W1p_z, W2m_z, W2p_z = get_predictions(pgc, file1, file2)

## I finally decided to use the median of 4 with no exclusion

In [12]:
W1_lst = W1m_i
W2_lst = W2m_i

### Averaging over various Random Forest regressors

N = len(W1m_i)
w1p_lst = np.zeros(N)
w2p_lst = np.zeros(N)


for j in range(N):
    wp_ = np.zeros(4)
    wp_[0]  = W1p_g[j]
    wp_[1]  = W1p_r[j]
    wp_[2]  = W1p_i[j]
    wp_[3]  = W1p_z[j]
    try:
        indx,   = np.where(wp_!=0)
        if len(indx)>0:
            wp_ = wp_[indx]
            w1p_lst[j] = np.median(wp_)
        else: 
            w1p_lst[j] = W1_lst[j]
    except:
        w1p_lst[j] = W1_lst[j]
        


for j in range(N):
    wp_ = np.zeros(4)
    wp_[0]  = W2p_g[j]
    wp_[1]  = W2p_r[j]
    wp_[2]  = W2p_i[j]
    wp_[3]  = W2p_z[j]
    try:
        indx,   = np.where(wp_!=0)
        if len(indx)>0:
            wp_ = wp_[indx]
            w2p_lst[j] = np.median(wp_)
        else: 
            w2p_lst[j] = W2_lst[j]
    except:
        w2p_lst[j] = W2_lst[j]


In [13]:
N = len(table['pgc'])
gamma_m_u = np.zeros(N)
gamma_p_u = np.zeros(N)
A_m_u = np.zeros(N)
A_p_u = np.zeros(N)

gamma_m_g = np.zeros(N)
gamma_p_g = np.zeros(N)
A_m_g = np.zeros(N)
A_p_g = np.zeros(N)

gamma_m_r = np.zeros(N)
gamma_p_r = np.zeros(N)
A_m_r = np.zeros(N)
A_p_r = np.zeros(N)

gamma_m_i = np.zeros(N)
gamma_p_i = np.zeros(N)
A_m_i = np.zeros(N)
A_p_i = np.zeros(N)

gamma_m_z = np.zeros(N)
gamma_p_z = np.zeros(N)
A_m_z = np.zeros(N)
A_p_z = np.zeros(N)

gamma_m_w1 = np.zeros(N)
gamma_p_w1 = np.zeros(N)
A_m_w1 = np.zeros(N)
A_p_w1 = np.zeros(N)

gamma_m_w2 = np.zeros(N)
gamma_p_w2 = np.zeros(N)
A_m_w2 = np.zeros(N)
A_p_w2 = np.zeros(N)

P0m = np.zeros(N)
P0p = np.zeros(N)

P0m_W1 = np.zeros(N)

for j in range(N):
    
    w2p    = w2p_lst[j]
    w2m    = W2_lst[j]
    w1m    = W1_lst[j]
    M21    = table["m21"][j]
    M0     = table["logWimx"][j]
    WBA    = table["Wba"][j]
    R50_W1 = table["R50_w1"][j]
    R50    = table["R50_w2"][j]
    WBAp   = table["Wbap"][j]
    R50p   = table["R50_w2p"][j]
    INC    = table["inc"][j]
    
    ## MEASURED
    if table["Wqlt"][j]>0:
        logWimx, c21w, mu50, P0  = get_PC(w2m, M21, M0, WBA, R50)
        _, _, _, P0_W1  = get_PC_W1(w1m, M21, M0, WBA, R50_W1)
        A_m_u[j],  gamma_m_u[j]  = redCorrect(INC, P0, band1='u', band2='w2', getGamma=True)
        A_m_g[j],  gamma_m_g[j]  = redCorrect(INC, P0, band1='g', band2='w2', getGamma=True)
        A_m_r[j],  gamma_m_r[j]  = redCorrect(INC, P0, band1='r', band2='w2', getGamma=True)
        A_m_i[j],  gamma_m_i[j]  = redCorrect(INC, P0, band1='i', band2='w2', getGamma=True)
        A_m_z[j],  gamma_m_z[j]  = redCorrect(INC, P0, band1='z', band2='w2', getGamma=True)
        A_m_w1[j], gamma_m_w1[j] = redCorrect(INC, P0, band1='w1', band2='w2', getGamma=True)
        A_m_w2[j], gamma_m_w2[j] = redCorrect(INC, P0, band1='w2', band2='w2', getGamma=True)
        P0m[j]=P0
        P0m_W1[j] = P0_W1
    
    ## PREDICTED
    if table["Sqlt"][j]>0:
        logWimx, c21w, mu50, P0 = get_PC(w2p, M21, M0, WBAp, R50p)
        A_p_u[j],  gamma_p_u[j]  = redCorrect(INC, P0, band1='u', band2='w2', getGamma=True)
        A_p_g[j],  gamma_p_g[j]  = redCorrect(INC, P0, band1='g', band2='w2', getGamma=True)
        A_p_r[j],  gamma_p_r[j]  = redCorrect(INC, P0, band1='r', band2='w2', getGamma=True)
        A_p_i[j],  gamma_p_i[j]  = redCorrect(INC, P0, band1='i', band2='w2', getGamma=True)
        A_p_z[j],  gamma_p_z[j]  = redCorrect(INC, P0, band1='z', band2='w2', getGamma=True)
        A_p_w1[j], gamma_p_w1[j] = redCorrect(INC, P0, band1='w1', band2='w2', getGamma=True)
        A_p_w2[j], gamma_p_w2[j] = redCorrect(INC, P0, band1='w2', band2='w2', getGamma=True)
        P0p[j]=P0

/home/ehsan/PanStarrs/Jan/HI/augment/reddening/redTools.py:710: RuntimeWarning: divide by zero encountered in log10
  we = 2.5*np.log10(2.*np.pi*(R50_w2*60)**2)-2.5*np.log10(Wba)
/home/ehsan/PanStarrs/Jan/HI/augment/reddening/redTools.py:710: RuntimeWarning: invalid value encountered in double_scalars
  we = 2.5*np.log10(2.*np.pi*(R50_w2*60)**2)-2.5*np.log10(Wba)
/home/ehsan/PanStarrs/Jan/HI/augment/reddening/redTools.py:695: RuntimeWarning: divide by zero encountered in log10
  we = 2.5*np.log10(2.*np.pi*(R50_w1*60)**2)-2.5*np.log10(Wba)
/home/ehsan/PanStarrs/Jan/HI/augment/reddening/redTools.py:695: RuntimeWarning: invalid value encountered in double_scalars
  we = 2.5*np.log10(2.*np.pi*(R50_w1*60)**2)-2.5*np.log10(Wba)
/home/ehsan/PanStarrs/Jan/HI/augment/reddening/redTools.py:466: RuntimeWarning: invalid value encountered in double_scalars
  gamma = (a*pc0**3+b*pc0**2+c*pc0+d)
/home/ehsan/PanStarrs/Jan/HI/augment/reddening/redTools.py:462: RuntimeWarning: invalid value encountered 

In [14]:
table['pc0'] = P0m
table['pc0p'] = P0p
table['pc0_w1'] = P0m_W1

Ec21w = np.sqrt(table['m21_e']**2+0.05**2)
table['Ec21w'] = Ec21w

w2_best = np.zeros(N)
R50_w2_best = np.zeros(N)
Wba_best = np.zeros(N)
for j in range(N):
    w2_best[j]     = W2_lst[j]
    R50_w2_best[j] = table["R50_w2"][j]
    Wba_best[j]    = table["Wba"][j]
    
    ## Use predicted values for effective radius and surface brightness if ...
    if (table["Wqlt"][j]<3 and table["Sqlt"][j]>1) or (table["Wqlt"][j]==0 and table["Sqlt"][j]>0):
        w2_best[j]     = w2p_lst[j]
        R50_w2_best[j] = table["R50_w2p"][j]
        Wba_best[j]    = table["Wbap"][j]     
        

## Measured - Predicted
c21w = table['m21'] - w2_best
table['c21w'] = c21w

mu50 = w2_best+2.5*np.log10(2.*np.pi*(R50_w2_best*60)**2)-2.5*np.log10(Wba_best)
table['mu50'] = mu50
dWba2 = ((0.1/6./R50_w2_best)**2)*(1+Wba_best**2)
c2 = (2.5/np.log(10))**2
Emu50 = np.sqrt(c2*(0.1/6./R50_w2_best)**2+c2*dWba2/Wba_best**2+0.05**2)
table['Emu50'] = Emu50 


logWimx_e = table['logWimx_e']
table['Epc0'] = np.sqrt((p0*logWimx_e/s[0])**2+(p1*Ec21w/s[1])**2+(p2*Emu50/s[2])**2)


table['u_s']  = table['u']-A_m_u
table['g_s']  = table['g']-A_m_g
table['r_s']  = table['r']-A_m_r
table['i_s']  = table['i']-A_m_i
table['z_s']  = table['z']-A_m_z
table['w1_s'] = table['w1']-A_m_w1
table['w2_s'] = table['w2']-A_m_w2

table['u_ss']  = table['u']-A_p_u
table['g_ss']  = table['g']-A_p_g
table['r_ss']  = table['r']-A_p_r
table['i_ss']  = table['i']-A_p_i
table['z_ss']  = table['z']-A_p_z
# table['w1_ss'] = table['w1']-A_p_w1
# table['w2_ss'] = table['w2']-A_p_w2

# indx, = np.where(table['Wqlt']==0)
# table['w1_ss'][indx] = w1p_lst[indx]-A_p_w1[indx]
# table['w2_ss'][indx] = w2p_lst[indx]-A_p_w2[indx]

table['w1_ss'] = w1p_lst-A_p_w1
table['w2_ss'] = w2p_lst-A_p_w2

/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in log10
/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in subtract
/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in divide
/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in divide
/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in sqrt
/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in sqrt


In [15]:
### Table modifications .... 

indx, = np.where(table['u000']<0)
table['u000'][indx] = 0
indx, = np.where(table['u']<0)
table['u'][indx] = 0

### NO SDSS data - NO prediction
indx, = np.where(table['Sqlt']==0)
table['u000'][indx]=0
table['g000'][indx]=0
table['r000'][indx]=0
table['i000'][indx]=0
table['z000'][indx]=0
table['u'][indx]=0
table['g'][indx]=0
table['r'][indx]=0
table['i'][indx]=0
table['z'][indx]=0
table['u_s'][indx]=0
table['g_s'][indx]=0
table['r_s'][indx]=0
table['i_s'][indx]=0
table['z_s'][indx]=0
table['u_ss'][indx]=0
table['g_ss'][indx]=0
table['r_ss'][indx]=0
table['i_ss'][indx]=0
table['z_ss'][indx]=0
table['w1_ss'][indx]=table['w1_s'][indx]
table['w2_ss'][indx]=table['w2_s'][indx]
A_p_u[indx]  = A_m_u[indx]
A_p_g[indx]  = A_m_g[indx]
A_p_r[indx]  = A_m_r[indx]
A_p_i[indx]  = A_m_i[indx]
A_p_z[indx]  = A_m_z[indx]
A_p_w1[indx] = A_m_w1[indx]
A_p_w2[indx] = A_m_w2[indx]
gamma_p_u[indx]  = gamma_m_u[indx]
gamma_p_g[indx]  = gamma_m_g[indx]
gamma_p_r[indx]  = gamma_m_r[indx]
gamma_p_i[indx]  = gamma_m_i[indx]
gamma_p_z[indx]  = gamma_m_z[indx]
gamma_p_w1[indx] = gamma_m_w1[indx]
gamma_p_w2[indx] = gamma_m_w2[indx]
w1p_lst[indx] = table['w1'][indx]
w2p_lst[indx] = table['w2'][indx]
table['pc0p'][indx] = table['pc0'][indx]

### NO WISE data
indx, = np.where(table['Wqlt']==0)
table['w1'][indx]=0
table['w2'][indx]=0
table['w1_s'][indx]=0
table['w2_s'][indx]=0
table['pc0'][indx] = table['pc0p'][indx]


indx, = np.where(w2p_lst == 0)
table['pc0p'][indx] = 0

indx, = np.where(W2_lst == 0)
table['pc0'][indx] = 0

## NO prediction 
indx, = np.where(w2p_lst == W2_lst)
table['pc0p'][indx] = table['pc0'][indx]
table['w1_ss'][indx]=table['w1_s'][indx]
table['w2_ss'][indx]=table['w2_s'][indx]
A_p_u[indx]  = A_m_u[indx]
A_p_g[indx]  = A_m_g[indx]
A_p_r[indx]  = A_m_r[indx]
A_p_i[indx]  = A_m_i[indx]
A_p_z[indx]  = A_m_z[indx]
A_p_w1[indx] = A_m_w1[indx]
A_p_w2[indx] = A_m_w2[indx]
gamma_p_u[indx]  = gamma_m_u[indx]
gamma_p_g[indx]  = gamma_m_g[indx]
gamma_p_r[indx]  = gamma_m_r[indx]
gamma_p_i[indx]  = gamma_m_i[indx]
gamma_p_z[indx]  = gamma_m_z[indx]
gamma_p_w1[indx] = gamma_m_w1[indx]
gamma_p_w2[indx] = gamma_m_w2[indx]
w1p_lst[indx] = table['w1'][indx]
w2p_lst[indx] = table['w2'][indx]

In [16]:
print len(table['inc'])
print len(gamma_p_i)
print len(P0m_W1)
print len(table['Wqlt'])

13535
13535
13535
13535


In [17]:
import copy
N = len(P0m_W1)

table['u_sss']  = copy.deepcopy(table['u_s'])
table['g_sss']  = copy.deepcopy(table['g_s'])
table['r_sss']  = copy.deepcopy(table['r_s'])
table['i_sss']  = copy.deepcopy(table['i_s'])
table['z_sss']  = copy.deepcopy(table['z_s'])
table['w1_sss'] = copy.deepcopy(table['w1_s'])
table['w2_sss'] = copy.deepcopy(table['w2_s'])
Reddening_Source = 2*np.ones(N)   # by default W2

table_   = np.genfromtxt('pgc_W2_poor_flags_W1.csv' , delimiter=',', filling_values=-1, names=True, dtype=None)
pgc_W2_poor_flags_W1 = table_['PGC']
table_   = np.genfromtxt('pgc_W2_poor_flags_P.csv'  , delimiter=',', filling_values=-1, names=True, dtype=None)
pgc_W2_poor_flags_P  = table_['PGC']

for i, idd in enumerate(pgc):
    
    if table['Wqlt'][i]==0:
        table['u_sss'][i]  = table['u_ss'][i]
        table['g_sss'][i]  = table['g_ss'][i]
        table['r_sss'][i]  = table['r_ss'][i]
        table['i_sss'][i]  = table['i_ss'][i]
        table['z_sss'][i]  = table['z_ss'][i]
        table['w1_sss'][i] = 0
        table['w2_sss'][i] = 0  
        Reddening_Source[i]=0
    else:
        if idd in pgc_W2_poor_flags_W1:  # use W1 predictions
            P0_w1 = P0m_W1[i]
            P0 = 1.021*P0_w1-0.094
            INC = table['inc'][i]
            table['u_sss'][i]  = table['u'][i]-redCorrect(INC, P0, band1='u', band2='w2')
            table['g_sss'][i]  = table['g'][i]-redCorrect(INC, P0, band1='g', band2='w2')
            table['r_sss'][i]  = table['r'][i]-redCorrect(INC, P0, band1='r', band2='w2')
            table['i_sss'][i]  = table['i'][i]-redCorrect(INC, P0, band1='i', band2='w2')
            table['z_sss'][i]  = table['z'][i]-redCorrect(INC, P0, band1='z', band2='w2')
            table['w1_sss'][i] = table['w1'][i]-redCorrect(INC, P0, band1='w1', band2='w2')
            table['w2_sss'][i] = table['w2'][i]-redCorrect(INC, P0, band1='w2', band2='w2')
            Reddening_Source[i]=1
        if idd in pgc_W2_poor_flags_P:   # use SDSS predictions
            table['u_sss'][i]  = table['u_ss'][i]
            table['g_sss'][i]  = table['g_ss'][i]
            table['r_sss'][i]  = table['r_ss'][i]
            table['i_sss'][i]  = table['i_ss'][i]
            table['z_sss'][i]  = table['z_ss'][i]
            table['w1_sss'][i] = table['w1_ss'][i]
            table['w2_sss'][i] = table['w2_ss'][i] 
            Reddening_Source[i]=0
        ## Otherwise the original values are good enough
            
    if table['Sqlt'][i]==0:
        table['u_sss'][i]  = 0
        table['g_sss'][i]  = 0
        table['r_sss'][i]  = 0
        table['i_sss'][i]  = 0
        table['z_sss'][i]  = 0

In [18]:
N = len(table['pgc'])
dummy = np.zeros(N)

indx, = np.where(table['Sqlt']>0)
dummy[indx]+=1
indx, = np.where(table['Wqlt']>0)
dummy[indx]+=1

indx, = np.where(table['inc']==0)
dummy[indx]=0   # remove if inc=0

indx, = np.where(dummy > 0)   # not face-on and some high quality data

myTable = Table()
myTable.add_column(Column(data=table['pgc'][indx], name='pgc'))
myTable.add_column(Column(data=table['Name'][indx], name='Name'))

myTable.add_column(Column(data=table['Vhel'][indx], name='Vhel', format='%0.1f'))
myTable.add_column(Column(data=table['Vls'][indx], name='Vls', format='%0.1f'))

myTable.add_column(Column(data=table['u000'][indx], name='u', format='%0.2f'))
myTable.add_column(Column(data=table['g000'][indx], name='g', format='%0.2f'))
myTable.add_column(Column(data=table['r000'][indx], name='r', format='%0.2f'))
myTable.add_column(Column(data=table['i000'][indx], name='i', format='%0.2f'))
myTable.add_column(Column(data=table['z000'][indx], name='z', format='%0.2f'))
myTable.add_column(Column(data=table['w100'][indx], name='w1', format='%0.2f'))
myTable.add_column(Column(data=table['w200'][indx], name='w2', format='%0.2f'))

myTable.add_column(Column(data=table['u'][indx], name='u_', format='%0.2f'))
myTable.add_column(Column(data=table['g'][indx], name='g_', format='%0.2f'))
myTable.add_column(Column(data=table['r'][indx], name='r_', format='%0.2f'))
myTable.add_column(Column(data=table['i'][indx], name='i_', format='%0.2f'))
myTable.add_column(Column(data=table['z'][indx], name='z_', format='%0.2f'))
myTable.add_column(Column(data=table['w1'][indx], name='w1_', format='%0.2f'))
myTable.add_column(Column(data=table['w2'][indx], name='w2_', format='%0.2f'))

myTable.add_column(Column(data=table['u_s'][indx], name='u_s', format='%0.2f'))
myTable.add_column(Column(data=table['g_s'][indx], name='g_s', format='%0.2f'))
myTable.add_column(Column(data=table['r_s'][indx], name='r_s', format='%0.2f'))
myTable.add_column(Column(data=table['i_s'][indx], name='i_s', format='%0.2f'))
myTable.add_column(Column(data=table['z_s'][indx], name='z_s', format='%0.2f'))
myTable.add_column(Column(data=table['w1_s'][indx], name='w1_s', format='%0.2f'))
myTable.add_column(Column(data=table['w2_s'][indx], name='w2_s', format='%0.2f'))

myTable.add_column(Column(data=A_m_u[indx], name='Au', format='%0.2f'))
myTable.add_column(Column(data=A_m_g[indx], name='Ag', format='%0.2f'))
myTable.add_column(Column(data=A_m_r[indx], name='Ar', format='%0.2f'))
myTable.add_column(Column(data=A_m_i[indx], name='Ai', format='%0.2f'))
myTable.add_column(Column(data=A_m_z[indx], name='Az', format='%0.2f'))
myTable.add_column(Column(data=A_m_w1[indx], name='Aw1', format='%0.2f'))
myTable.add_column(Column(data=A_m_w2[indx], name='Aw2', format='%0.2f'))

myTable.add_column(Column(data=gamma_m_u[indx], name='gamma_u', format='%0.2f'))
myTable.add_column(Column(data=gamma_m_g[indx], name='gamma_g', format='%0.2f'))
myTable.add_column(Column(data=gamma_m_r[indx], name='gamma_r', format='%0.2f'))
myTable.add_column(Column(data=gamma_m_i[indx], name='gamma_i', format='%0.2f'))
myTable.add_column(Column(data=gamma_m_z[indx], name='gamma_z', format='%0.2f'))
myTable.add_column(Column(data=gamma_m_w1[indx], name='gamma_w1', format='%0.2f'))
myTable.add_column(Column(data=gamma_m_w2[indx], name='gamma_w2', format='%0.2f'))


# myTable.add_column(Column(data=W1_lst[indx], name='W1m_', format='%0.2f'))
# myTable.add_column(Column(data=W2_lst[indx], name='W2m_', format='%0.2f'))
# myTable.add_column(Column(data=w1p_lst[indx], name='W1p_', format='%0.2f'))
myTable.add_column(Column(data=w2p_lst[indx], name='W2p_', format='%0.2f'))

myTable.add_column(Column(data=table['u_ss'][indx], name='u_ss', format='%0.2f'))
myTable.add_column(Column(data=table['g_ss'][indx], name='g_ss', format='%0.2f'))
myTable.add_column(Column(data=table['r_ss'][indx], name='r_ss', format='%0.2f'))
myTable.add_column(Column(data=table['i_ss'][indx], name='i_ss', format='%0.2f'))
myTable.add_column(Column(data=table['z_ss'][indx], name='z_ss', format='%0.2f'))
# myTable.add_column(Column(data=table['w1_ss'][indx], name='w1_ss', format='%0.2f'))
myTable.add_column(Column(data=table['w2_ss'][indx], name='w2_ss', format='%0.2f'))



myTable.add_column(Column(data=A_p_u[indx], name='A_p_u', format='%0.2f'))
myTable.add_column(Column(data=A_p_g[indx], name='A_p_g', format='%0.2f'))
myTable.add_column(Column(data=A_p_r[indx], name='A_p_r', format='%0.2f'))
myTable.add_column(Column(data=A_p_i[indx], name='A_p_i', format='%0.2f'))
myTable.add_column(Column(data=A_p_z[indx], name='A_p_z', format='%0.2f'))
myTable.add_column(Column(data=A_p_w1[indx], name='A_p_w1', format='%0.2f'))
myTable.add_column(Column(data=A_p_w2[indx], name='A_p_w2', format='%0.2f'))

myTable.add_column(Column(data=gamma_p_u[indx], name='gamma_p_u', format='%0.2f'))
myTable.add_column(Column(data=gamma_p_g[indx], name='gamma_p_g', format='%0.2f'))
myTable.add_column(Column(data=gamma_p_r[indx], name='gamma_p_r', format='%0.2f'))
myTable.add_column(Column(data=gamma_p_i[indx], name='gamma_p_i', format='%0.2f'))
myTable.add_column(Column(data=gamma_p_z[indx], name='gamma_p_z', format='%0.2f'))
myTable.add_column(Column(data=gamma_p_w1[indx], name='gamma_p_w1', format='%0.2f'))
myTable.add_column(Column(data=gamma_p_w2[indx], name='gamma_p_w2', format='%0.2f'))

myTable.add_column(Column(data=table['m21'][indx], name='m21', format='%0.3f'))
myTable.add_column(Column(data=table['m21_e'][indx], name='m21_e', format='%0.3f'))

myTable.add_column(Column(data=table['Wmx'][indx], name='Wmx', format='%0.3f'))
myTable.add_column(Column(data=table['eWmx'][indx], name='eWmx', format='%0.3f'))


myTable.add_column(Column(data=table['logWimx'][indx], name='logWimx', format='%0.3f'))
myTable.add_column(Column(data=table['logWimx_e'][indx], name='logWimx_e', format='%0.3f'))                   
myTable.add_column(Column(data=table['c21w'][indx], name='c21w', format='%0.2f'))
myTable.add_column(Column(data=table['Ec21w'][indx], name='c21w_e', format='%0.2f'))                            
myTable.add_column(Column(data=table['mu50'][indx], name='mu50', format='%0.2f'))
myTable.add_column(Column(data=table['Emu50'][indx], name='mu50_e', format='%0.2f'))                     
myTable.add_column(Column(data=table['pc0'][indx], name='P0_w2', format='%0.2f'))
myTable.add_column(Column(data=table['pc0p'][indx], name='P0_w2p', format='%0.2f'))
myTable.add_column(Column(data=table['Epc0'][indx], name='P0_w2_e', format='%0.2f')) 
myTable.add_column(Column(data=table['pc0_w1'][indx], name='P0_w1', format='%0.2f'))
myTable.add_column(Column(data=table['inc'][indx], name='inc'))
myTable.add_column(Column(data=table['inc_e'][indx], name='inc_e'))
myTable.add_column(Column(data=R50_w2_best[indx], name='R50', format='%0.2f'))
myTable.add_column(Column(data=Wba_best[indx], name='Wba', format='%0.2f')) 

myTable.add_column(Column(data=table['Sqlt'][indx], name='Sqlt', dtype=np.dtype(int)))
myTable.add_column(Column(data=table['Wqlt'][indx], name='Wqlt', dtype=np.dtype(int)))
######################################################################################
myTable.add_column(Column(data=table['u_sss'][indx], name='u_sss', format='%0.2f'))
myTable.add_column(Column(data=table['g_sss'][indx], name='g_sss', format='%0.2f'))
myTable.add_column(Column(data=table['r_sss'][indx], name='r_sss', format='%0.2f'))
myTable.add_column(Column(data=table['i_sss'][indx], name='i_sss', format='%0.2f'))
myTable.add_column(Column(data=table['z_sss'][indx], name='z_sss', format='%0.2f'))
myTable.add_column(Column(data=table['w1_sss'][indx], name='w1_sss', format='%0.2f'))
myTable.add_column(Column(data=table['w2_sss'][indx], name='w2_sss', format='%0.2f'))
myTable.add_column(Column(data=Reddening_Source[indx], name='R_source', dtype=np.dtype(int)))
######################################################################################

myTable.write('corrected_mags_linewidth_all.csv', format='ascii.fixed_width',delimiter='|', bookend=False, overwrite=True) 
